In [ ]:
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
# from xnoah.data_matrix import stack_cat, unstack_cat
# import holoviews as hv
# hv.extension('bokeh')
data = xr.open_dataset("../data/processed/regridded.nc", chunks={'t':100}).isel(t=slice(0,300))

In [ ]:
T = data['T'].transpose('t', 'x', 'z').data

In [ ]:
import tensorflow as tf

In [ ]:
nt, nx, nz= T.shape

In [ ]:
from tensorflow.contrib.opt import ScipyOptimizerInterface

x = tf.placeholder("float")
y = tf.placeholder("float")
w = tf.Variable(np.zeros((30, nz, nz)), name='weight', dtype="float")
b = tf.Variable(np.zeros((nz)), name='bias', dtype="float")

pred = tf.nn.conv1d(x, w, stride=1, padding='SAME', data_format="NHWC") + b


# loss function
mse = tf.reduce_sum(tf.pow(pred-y, 2))

# mse
optim = ScipyOptimizerInterface(mse, [w, b], options={'maxiter': 10})

init = tf.global_variables_initializer()

In [ ]:

with tf.Session() as sess:
    sess.run(init)
    optim.minimize(sess, feed_dict={x: T[:-3], y:T[3:]})
    wo, bo= sess.run([w,b])

In [ ]:
import holoviews as hv
hv.extension("bokeh")

In [ ]:
wo.shape

In [ ]:
d =xr.DataArray(wo, dims=['x', 'z', 'zp'])

In [ ]:
ds = hv.Dataset((np.arange(64), np.arange(64), np.arange(30), wo), kdims=['zp', 'z', 'x'], vdims=['var'])

In [ ]:
ds

In [ ]:
ds.to(hv.Image, ["z", "zp"])

In [ ]:
ds.to(hv.Image, ["x", "zp"])